# Processing training data from raw files

# Notebook n°1

In [2]:
basepath = '/data/conda/recnn/data'

In [3]:
%load_ext cython
import numpy as np
import h5py
import multiprocessing as mp
import sys
sys.path.append("..")
import pickle
from functools import partial
from recnn.preprocessing import randomize
from recnn.preprocessing import sequentialize_by_pt
import copy
from recnn.preprocessing import _pt
from rootpy.vector import LorentzVector
from recnn.preprocessing import multithreadmap
from functools import partial

Welcome to JupyROOT 6.10/08


In [4]:
%%cython -f -+ -I/usr/local/include --link-args=-Wl,-rpath,/usr/local/lib -lm -L/usr/local/lib -lfastjettools -lfastjet -lfastjetplugins -lsiscone_spherical -lsiscone
import numpy as np
cimport numpy as np
np.import_array()

from libcpp.pair cimport pair
from libcpp.vector cimport vector

cdef extern from "/home/yohann/Desktop/stage/recnn/notebooks/fj.cc":
    void fj(vector[double]& a, 
            vector[vector[int]]& trees, 
            vector[vector[double]]& contents, 
            vector[double]& masses, 
            vector[double]& pts, 
            double R, int jet_algorithm)
    
cpdef cluster(np.ndarray[np.double_t, ndim=2, mode="c"] a, 
              R=1.0, jet_algorithm=0):
    cdef vector[double] v
    cdef vector[vector[int]] trees
    cdef vector[vector[double]] contents
    cdef vector[double] masses
    cdef vector[double] pts 
    for value in a.ravel():
        v.push_back(value)
    
    fj(v, trees, contents, masses, pts, R=R, jet_algorithm=jet_algorithm)
    jets = []
    
    for tree, content, mass, pt in zip(trees, contents, masses, pts):
        tree = np.array(tree).reshape(-1, 2)
        content = np.array(content).reshape(-1, 4)
        jets.append((tree, content, mass, pt))
        
    return jets

cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
In file included from /opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1809:0,
                 from /opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/yohann/.cache/ipython/cython/_cython_magic_89e3fcb72e2ea30883154e4f31d44188.cpp:535:
/opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^


In [5]:
def cast(event, soft=0):
    a = np.zeros((len(event)+soft, 4))
    
    for i, p in enumerate(event):
        a[i, 3] = p[0]
        a[i, 0] = p[1]
        a[i, 1] = p[2]
        a[i, 2] = p[3]
    
    # sprinkle soft particles
    for i in range(len(event), len(event)+soft):
        v = LorentzVector()
        v.set_pt_eta_phi_m(10e-5, np.random.rand() * 10 - 5, np.random.rand() * 2 * np.pi, 0.0)
        a[i, 0] = v.px
        a[i, 1] = v.py
        a[i, 2] = v.pz
        a[i, 3] = v.e
    return a


In [6]:
def ff(e):
    t=cast(e, soft=0)
    tree, content, mass, pt = cluster(t, jet_algorithm=1)[0]  # dump highest pt jet only
    jet = {}
    
    jet["root_id"] = 0
    jet["tree"] = tree
    jet["content"] = content
    jet["mass"] = mass
    jet["pt"] = pt
    jet["energy"] = content[0, 3]

    px = content[0, 0]
    py = content[0, 1]
    pz = content[0, 2]
    p = (content[0, 0:3] ** 2).sum() ** 0.5
    eta = 0.5 * (np.log(p + pz) - np.log(p - pz))
    phi = np.arctan2(py, px)
    
    jet["eta"] = eta
    jet["phi"] = phi
    
    return(jet)

In [7]:
events = np.array(np.load(basepath+'/QCD_Pt80to120_ext2_dataformat_processed_for_recnn.npy'))
background = multithreadmap(ff,events)

events = np.array(np.load(basepath+'/HiggsSUSYGG160_dataformat_processed_for_recnn.npy'))
signal = multithreadmap(ff,events)


#--------------------------------------------------------------------------
#                         FastJet release 3.0.6
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the terms of the GNU GPLv2.
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#

# Notebook n°2

## W vs QCD

In [8]:
X = np.array(background[:100000]+signal[:100000])
y = np.array([0]*100000+[1]*100000)

# Notebook n°3

### preprocessing function

In [9]:
%%cython -f -+ -I/usr/local/include --link-args=-Wl,-rpath,/usr/local/lib -lm -L/usr/local/lib -lfastjettools -lfastjet -lfastjetplugins -lsiscone_spherical -lsiscone
import numpy as np
cimport numpy as np
np.import_array()

from libcpp.pair cimport pair
from libcpp.vector cimport vector

cdef extern from "/home/yohann/Desktop/stage/recnn/notebooks/fj.cc":
    void fj(vector[double]& a, 
            vector[vector[int]]& trees, 
            vector[vector[double]]& contents, 
            vector[double]& masses, 
            vector[double]& pts, 
            double R, int jet_algorithm)
    
cpdef cluster(np.ndarray[np.double_t, ndim=2, mode="c"] a, 
              R=0.3, jet_algorithm=0):
    cdef vector[double] v
    cdef vector[vector[int]] trees
    cdef vector[vector[double]] contents
    cdef vector[double] masses
    cdef vector[double] pts 
    for value in a.ravel():
        v.push_back(value)
    
    fj(v, trees, contents, masses, pts, R=R, jet_algorithm=jet_algorithm)
    jets = []
    
    for tree, content, mass, pt in zip(trees, contents, masses, pts):
        tree = np.array(tree).reshape(-1, 2)
        content = np.array(content).reshape(-1, 4)
        jets.append((tree, content, mass, pt))
        
    return jets

cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
In file included from /opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1809:0,
                 from /opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/yohann/.cache/ipython/cython/_cython_magic_f5b325c9c9f5dc1aed38a68917da0cab.cpp:535:
/opt/deeplearning/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^


In [10]:
# Preprocessing algorithm:
# 1. j = the highest pt anti-kt jet (R=1)
# 2. run kt (R=0.3) on the constituents c of j, resulting in subjets sj1, sj2, ..., sjN
# 3. phi = sj1.phi(); for all c, do c.rotate_z(-phi)
# 4. bv = sj1.boost_vector(); bv.set_perp(0); for all c, do c.boost(-bv)
# 5. deltaz = sj1.pz - sj2.pz; deltay = sj1.py - sj2.py; alpha = -atan2(deltaz, deltay); for all c, do c.rotate_x(alpha)
# 6. if sj3.pz < 0: for all c, do c.set_pz(-c.pz)
# 7. finally recluster all transformed constituents c into a single jet (using kt or anti-kt? r?)

def preprocess(jet, output="kt", colinear_splits=0, trimming=0.0):
    jet = copy.deepcopy(jet)
    constituents = jet["content"][jet["tree"][:, 0] == -1]
    for i in range(colinear_splits):
        #j = np.random.randint(len(constituents))
        j = np.argmax([_pt(c) for c in constituents])
        v = LorentzVector(constituents[j])
        eps = np.random.rand()
        
        p1 = LorentzVector()
        p2 = LorentzVector()
        p1.set_pt_eta_phi_m(v.pt() * eps, v.eta(), v.phi(), v.m() * eps ** 0.5)
        p2.set_pt_eta_phi_m(v.pt() * (1. - eps), v.eta(), v.phi(), 0.0)

        constituents[j][0] = p1.px
        constituents[j][1] = p1.py
        constituents[j][2] = p1.pz
        constituents[j][3] = p1.e
        
        constituents = np.vstack([constituents, 
                                  np.array([[p2.px, p2.py, p2.pz, p2.e]])])

    # run kt (R=0.3) on the constituents c of j, resulting in subjets sj1, sj2, ..., sjN
    subjets = cluster(constituents, R=0.3, jet_algorithm=0)

    # trimming
    if trimming > 0.0:
        subjets = [(tree, content, mass, pt) for tree, content, mass, pt in subjets if pt > trimming * jet["pt"]]
    else:
        subjets = [(tree, content, mass, pt) for tree, content, mass, pt in subjets]

    # phi = sj1.phi()
    # for all c, do c.rotate_z(-phi)
    v = subjets[0][1][0]
    v = LorentzVector(v)

    phi = v.phi()
    
    for _, content, _, _ in subjets:
        for i in range(len(content)):
            v = LorentzVector(content[i])
            v.rotate_z(-phi)
            content[i, 0] = v[0]
            content[i, 1] = v[1]
            content[i, 2] = v[2]
            content[i, 3] = v[3]

    # bv = sj1.boost_vector()
    # bv.set_perp(0)
    # for all c, do c.boost(-bv)
    v = subjets[0][1][0]
    v = LorentzVector(v)
    bv = v.boost_vector()
    bv.set_perp(0)
    for _, content, _, _ in subjets:
        for i in range(len(content)):
            v = LorentzVector(content[i])
            v.boost(-bv)
            content[i, 0] = v[0]
            content[i, 1] = v[1]
            content[i, 2] = v[2]
            content[i, 3] = v[3]
    # deltaz = sj1.pz - sj2.pz
    # deltay = sj1.py - sj2.py
    # alpha = -atan2(deltaz, deltay)
    # for all c, do c.rotate_x(alpha)
    if len(subjets) >= 2:
        deltaz = subjets[0][1][0, 2] - subjets[1][1][0, 2]
        deltay = subjets[0][1][0, 1] - subjets[1][1][0, 1]
        alpha = -np.arctan2(deltaz, deltay)
        for _, content, _, _ in subjets:
            for i in range(len(content)):
                v = LorentzVector(content[i])
                v.rotate_x(alpha)
                content[i, 0] = v[0]
                content[i, 1] = v[1]
                content[i, 2] = v[2]
                content[i, 3] = v[3]
    # if sj3.pz < 0: for all c, do c.set_pz(-c.pz)
    if len(subjets) >= 3 and subjets[2][1][0, 2] < 0:
        for _, content, _, _ in subjets:
            for i in range(len(content)):
                content[i, 2] *= -1.0
                
    # finally recluster all transformed constituents c into a single jet 
    constituents = []
    
    for tree, content, _, _ in subjets:
        constituents.append(content[tree[:, 0] == -1])
        
    constituents = np.vstack(constituents)

    if output == "anti-kt":
        subjets = cluster(constituents, R=100., jet_algorithm=1)
    elif output == "kt":
        subjets = cluster(constituents, R=100., jet_algorithm=0)
    elif output == "cambridge":
        subjets = cluster(constituents, R=100., jet_algorithm=2)
    else:
        raise
    
    
    jet["tree"] = subjets[0][0]
    jet["content"] = subjets[0][1]
    
    v = LorentzVector(jet["content"][0])
    jet["phi"] = v.phi()
    jet["eta"] = v.eta()
    jet["energy"] = v.E()
    jet["mass"] = v.m()
    jet["pt"] = v.pt()
    jet["root_id"] = 0
    
    return jet

### Convert data

In [11]:
f = basepath+'/picklefiles/preprocessed_training_set'

length = len(X)//2

In [12]:
#Save all versions of the dataset

#anti-kt
X_,y_ = np.copy(X[np.random.permutation(len(X))]),np.copy(y[np.random.permutation(len(X))])

X_ = multithreadmap(preprocess,X_,output='anti-kt')


X_test, y_test = X_[length:],y_[length:]
X_train, y_train = X_[:length],y_[:length]

fd = open(f+"_anti-kt_test.pickle", "wb")
pickle.dump((X_test, y_test), fd, protocol=2)
fd.close()

fd = open(f+"_anti-kt_train.pickle", "wb")
pickle.dump((X_train, y_train), fd, protocol=2)
fd.close()


#--------------------------------------------------------------------------
#                         FastJet release 3.0.6
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the terms of the GNU GPLv2.
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#

In [13]:
#kt
X_,y_ = np.copy(X[np.random.permutation(len(X))]),np.copy(y[np.random.permutation(len(X))])


X_ = multithreadmap(preprocess,X_,output='kt')


X_test, y_test = X_[length:],y_[length:]
X_train, y_train = X_[:length],y_[:length]

fd = open(f+"_kt_test.pickle", "wb")
pickle.dump((X_test, y_test), fd, protocol=2)
fd.close()

fd = open(f+"_kt_train.pickle", "wb")
pickle.dump((X_train, y_train), fd, protocol=2)
fd.close()

#--------------------------------------------------------------------------
#                         FastJet release 3.0.6
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the terms of the GNU GPLv2.
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#

In [14]:
#random
X_,y_ = np.copy(X[np.random.permutation(len(X))]),np.copy(y[np.random.permutation(len(X))])


X_=multithreadmap(randomize,multithreadmap(preprocess,X_,output="anti-kt"))

X_test, y_test = X_[length:],y_[length:]
X_train, y_train = X_[:length],y_[:length]

fd = open(f+"_random_test.pickle", "wb")
pickle.dump((X_test, y_test), fd, protocol=2)
fd.close()

fd = open(f+"_random_train.pickle", "wb")
pickle.dump((X_train, y_train), fd, protocol=2)
fd.close()

#--------------------------------------------------------------------------
#                         FastJet release 3.0.6
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the terms of the GNU GPLv2.
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#

In [15]:
#seq by pt
X_,y_ = np.copy(X[np.random.permutation(len(X))]),np.copy(y[np.random.permutation(len(X))])

X_=multithreadmap(sequentialize_by_pt,multithreadmap(preprocess,X_,output="anti-kt"),reverse=False)

X_test, y_test = X_[length:],y_[length:]
X_train, y_train = X_[:length],y_[:length]

fd = open(f+"_seqpt_test.pickle", "wb")
pickle.dump((X_test, y_test), fd, protocol=2)
fd.close()

fd = open(f+"_seqpt_train.pickle", "wb")
pickle.dump((X_train, y_train), fd, protocol=2)
fd.close()

#--------------------------------------------------------------------------
#                         FastJet release 3.0.6
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the terms of the GNU GPLv2.
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#

In [16]:
#seq by pt reversed
X_,y_ = np.copy(X[np.random.permutation(len(X))]),np.copy(y[np.random.permutation(len(X))])

X_=multithreadmap(sequentialize_by_pt,multithreadmap(preprocess,X_,output="anti-kt"),reverse=True)

X_test, y_test = X_[length:],y_[length:]
X_train, y_train = X_[:length],y_[:length]

fd = open(f+"_seqpt_reversed_test.pickle", "wb")
pickle.dump((X_test, y_test), fd, protocol=2)
fd.close()

fd = open(f+"_seqpt_reversed_train.pickle", "wb")
pickle.dump((X_train, y_train), fd, protocol=2)
fd.close()

#--------------------------------------------------------------------------
#                         FastJet release 3.0.6
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the terms of the GNU GPLv2.
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#